<a href="https://colab.research.google.com/github/neerajguleria1/driver-drowsiness-detection/blob/main/recreate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd

np.random.seed(42)
n=2000

fatigue=np.random.randint(0,11,n)

alertness=np.clip(
    1-(fatigue/12)+np.random.normal(0,0.05,n),
    0.05,
    1.0
)

heart_rate=np.random.normal(
    80+fatigue*2-alertness*5,
    10,
    n
).clip(50,150).astype(int)

speed=np.random.normal(
    120-fatigue*3,
    30,
    n
).clip(30,250).astype(int)

seatbelt=np.random.choice([0,1],size=n,p=[0.3,0.7])

df=pd.DataFrame({
    "Speed":speed,
    "Seatbelt":seatbelt,
    "Alertness":alertness,
    "HR":heart_rate,
    "Fatigue":fatigue
})

df["prev_alertness"]=df["Alertness"].shift(1)
df["prev_alertness"]=(df["prev_alertness"].fillna(df["Alertness"]))

df["speed_change"]=df["Speed"].diff().abs()
df["speed_change"]=df["speed_change"].fillna(0)

df["Fatigue_norm"]=df["Fatigue"]/10

drowsy_prob=(
    0.35*(1-df["Alertness"])
    +0.30*df["Fatigue_norm"]
    +0.10*(df["HR"]>105).astype(int)
    +0.05*(df["HR"]<55).astype(int)
    +0.08*(df["Speed"]<50).astype(int)
    +0.07*(df["speed_change"]<8).astype(int)
    +0.05*((df["prev_alertness"]-df["Alertness"])>0.2).astype(int)
).clip(0,1)

df["Drowsy"]=np.random.binomial(1,drowsy_prob)

df.to_csv("driver_drowsiness_training.csv",index=False)
print("Dataset created successfully")
print(df.head())

Dataset created successfully
   Speed  Seatbelt  Alertness  HR  Fatigue  prev_alertness  speed_change  \
0     94         1   0.531937  94        6        0.531937           0.0   
1    108         0   0.692850  78        3        0.531937          14.0   
2    110         1   0.248338  97       10        0.692850           2.0   
3     78         0   0.359349  99        7        0.248338          32.0   
4     99         1   0.681798  65        4        0.359349          21.0   

   Fatigue_norm  Drowsy  
0           0.6       1  
1           0.3       0  
2           1.0       1  
3           0.7       1  
4           0.4       0  
